In [70]:
inputPath = 'abfss://container1@553701adls.dfs.core.windows.net/inputfile'

StatementMeta(SparkPool1, 1, 31, Finished, Available)

In [71]:
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

inputSchema = StructType([
  StructField("CustomerID", IntegerType(), True),
  StructField("CustomerAddress", StringType(), True)  
])


rawDataDF = (spark.read
  .option("header", "true")
  .schema(inputSchema)
  .csv(inputPath)
            )

StatementMeta(SparkPool1, 1, 32, Finished, Available)

In [72]:
display(rawDataDF)

StatementMeta(SparkPool1, 1, 33, Finished, Available)

SynapseWidget(Synapse.DataFrame, 99918fbc-97f2-4b12-ae75-7bd950280772)

In [ ]:
rawDataDF.describe()


In [73]:
DataPath = 'abfss://container1@553701adls.dfs.core.windows.net/deltafile'

StatementMeta(SparkPool1, 1, 34, Finished, Available)

In [90]:
# write to Delta Lake
rawDataDF.write.mode("overwrite").format("delta").save(DataPath)

StatementMeta(SparkPool1, 1, 51, Finished, Available)

In [91]:
from pyspark.sql.functions import *
updatedDF = rawDataDF.withColumn('Valid_To', to_timestamp(lit('12-24-2099 12:01:19.000'),'MM-dd-yyyy HH:mm:ss.SSSS').cast("timestamp"))\
.withColumn('Valid_From',F.current_timestamp().cast("timestamp"))\
.withColumn("CustomerSK", row_number().over(Window.orderBy('CustomerID')))


  

StatementMeta(SparkPool1, 1, 52, Finished, Available)

In [110]:
updatedDF.write.mode("overwrite").format("delta").option("mergeSchema","true").save(DataPath)

StatementMeta(SparkPool1, 1, 71, Finished, Available)

In [117]:
spark.sql("""
  DROP TABLE IF EXISTS customer_data
""")
spark.sql("""
  CREATE TABLE customer_data
  USING DELTA
  LOCATION '{}'
""".format(DataPath))

StatementMeta(SparkPool1, 1, 78, Finished, Available)

DataFrame[]

In [114]:
updatedDF.createOrReplaceTempView("customer_data")

StatementMeta(SparkPool1, 1, 75, Finished, Available)

In [115]:
%%sql
DESCRIBE customer_data

StatementMeta(SparkPool1, 1, 76, Finished, Available)

<Spark SQL result set with 5 rows and 3 fields>

In [93]:
%%sql
select * from customer_data

StatementMeta(SparkPool1, 1, 54, Finished, Available)

<Spark SQL result set with 5 rows and 5 fields>

In [78]:
newFile = 'abfss://container1@553701adls.dfs.core.windows.net/newfile'
newDataDF = (spark
  .read
  .option("header", "true")
  .schema(inputSchema)
  .csv(newFile))

StatementMeta(SparkPool1, 1, 39, Finished, Available)

In [79]:
display(newDataDF)

StatementMeta(SparkPool1, 1, 40, Finished, Available)

SynapseWidget(Synapse.DataFrame, a03c6f30-3d69-4b93-9075-150345afaacd)

In [104]:
# write to Delta Lake
newDataDF.write.mode("overwrite").format("delta")

StatementMeta(SparkPool1, 1, 65, Finished, Available)

In [105]:
newDataDF.createOrReplaceTempView("new_customer_data")

StatementMeta(SparkPool1, 1, 66, Finished, Available)

In [100]:
%%sql
select * from new_customer_data

StatementMeta(SparkPool1, 1, 61, Finished, Available)

<Spark SQL result set with 3 rows and 2 fields>

In [106]:
%%sql
 SELECT
null AS CustomerSK, CustomerID, CustomerAddress, current_timestamp() AS Valid_From, null AS Valid_To
FROM new_customer_data
UNION ALL
SELECT
CustomerSK, CustomerID, CustomerAddress, Valid_From, Valid_To
FROM customer_data
WHERE CustomerID IN
(SELECT CustomerID FROM new_customer_data)


StatementMeta(SparkPool1, 1, 67, Finished, Available)

<Spark SQL result set with 4 rows and 5 fields>

In [118]:
%%sql
MERGE INTO customer_data
USING
 
-- Update table with rows to match (new and old referenced as seen in example above)
( SELECT
null AS CustomerSK, CustomerID, CustomerAddress, current_timestamp() AS Valid_From, null AS Valid_To
FROM new_customer_data
UNION ALL
SELECT
CustomerSK, CustomerID, CustomerAddress, Valid_From, Valid_To
FROM customer_data
WHERE CustomerID IN
(SELECT CustomerID FROM new_customer_data)
) ChangeRows
 
-- based on the following column(s)
ON customer_data.CustomerSK = ChangeRows.CustomerSK
 
-- if there is a match do this…
WHEN MATCHED THEN
UPDATE SET customer_data.Valid_To = current_timestamp()
 
-- if there is no match insert new row
WHEN NOT MATCHED THEN INSERT *

StatementMeta(SparkPool1, 1, 79, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [119]:
%%sql
select * from customer_data

StatementMeta(SparkPool1, 1, 80, Finished, Available)

<Spark SQL result set with 8 rows and 5 fields>

In [123]:
%%sql
SELECT  ROW_NUMBER() OVER (ORDER BY CustomerSK NULLS LAST)  AS CustomerSK 
FROM customer_data

StatementMeta(SparkPool1, 1, 84, Finished, Available)

<Spark SQL result set with 8 rows and 1 fields>

In [145]:
%%sql
SELECT  ROW_NUMBER() OVER (ORDER BY CustomerSK NULLS LAST)  AS CustomerSK
FROM customer_data

StatementMeta(SparkPool1, 1, 105, Finished, Available)

<Spark SQL result set with 8 rows and 1 fields>

In [156]:
df = sqlContext.table("customer_data")

StatementMeta(SparkPool1, 1, 116, Finished, Available)

In [157]:
display(df)

StatementMeta(SparkPool1, 1, 117, Finished, Available)

SynapseWidget(Synapse.DataFrame, 0a0851c9-c37b-4ffb-b8eb-0d89f0f6984e)

In [155]:
%%sql
Select * from customer_data

StatementMeta(SparkPool1, 1, 115, Finished, Available)

<Spark SQL result set with 8 rows and 5 fields>

In [160]:
df.selectExpr(
"ROW_NUMBER() OVER (ORDER BY CustomerSK NULLS LAST)  AS CustomerSK", 
"CustomerID", "CustomerAddress", "Valid_From", "Valid_To"
).show()

StatementMeta(SparkPool1, 1, 120, Finished, Available)

+----------+----------+--------------------+--------------------+--------------------+
|CustomerSK|CustomerID|     CustomerAddress|          Valid_From|            Valid_To|
+----------+----------+--------------------+--------------------+--------------------+
|         1|         1|82 Margate Drive,...|2022-05-28 07:00:...| 2099-12-24 12:01:19|
|         2|         2|135 High Barns, E...|2022-05-28 07:00:...| 2099-12-24 12:01:19|
|         3|         3|Guyzance Cottage,...|2022-05-28 07:00:...| 2099-12-24 12:01:19|
|         4|         4|322 Fernhill, Mou...|2022-05-28 07:00:...| 2099-12-24 12:01:19|
|         5|         5|381 Southborough ...|2022-05-28 07:00:...|2022-05-28 07:16:...|
|         6|         5|34 Park Road, Eas...|2022-05-28 07:16:...|                null|
|         7|         6|169 Manchester Ro...|2022-05-28 07:16:...|                null|
|         8|         7|52 Broadway, Plym...|2022-05-28 07:16:...|                null|
+----------+----------+--------------------

In [161]:
df.createOrReplaceTempView("customer_data")

StatementMeta(SparkPool1, 1, 121, Finished, Available)

In [162]:
%%sql
Select * from customer_data

StatementMeta(SparkPool1, 1, 122, Finished, Available)

<Spark SQL result set with 8 rows and 5 fields>